<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Financial Data Science &mdash; SQL DBs

**DataNatives Berlin 2015**

Dr Yves J Hilpisch

<a href='mailto:team@tpq.io'>team@tpq.io</a> | <a href='http://tpq.io'>http://tpq.io</a>

The Python Quants GmbH

## Importing & Database Connection

This notebook is about the `ibis` project. See the tutorial under [http://blog.ibis-project.org/sqlite-crunchbase-quickstart/](http://blog.ibis-project.org/sqlite-crunchbase-quickstart/).

In [ ]:
# getting the database to work with
!wget https://ibis-resources.s3.amazonaws.com/data/crunchbase/crunchbase.db

In [ ]:
import ibis
ibis.options.interactive = True

In [ ]:
con = ibis.sqlite.connect('crunchbase.db')

## Basic Operations and Lookups

In [ ]:
con.list_tables()

In [ ]:
rounds = con.table('rounds')
rounds.info()

In [ ]:
rounds.funding_round_type.value_counts()

In [ ]:
acquisitions = con.table('acquisitions')
expr = (acquisitions.price_amount
        .isnull()
        .name('has_price')
        .value_counts())
expr

In [ ]:
df = expr.execute()
df

In [ ]:
type(expr)

In [ ]:
companies = con.table('companies')

expr = companies.funding_total_usd.mean()
type(expr)

In [ ]:
expr.execute()

## Funding Metrics

In [ ]:
funded_at = rounds.funded_at.cast('timestamp')
funded_at.year().value_counts()

In [ ]:
rounds.funding_round_code.value_counts()

In [ ]:
year = funded_at.year().name('year')

expr = (rounds[(rounds.funding_round_type == 'venture') &
               year.between(2000, 2015) &
               rounds.funding_round_code.notnull()]
        .group_by([year, 'funding_round_code'])
        .size())

results = expr.execute()
results[:10]

In [ ]:
pivoted = (results.set_index(['year', 'funding_round_code'])
           .unstack('funding_round_code')
           .fillna(0))
pivoted

In [ ]:
funding_buckets = [0, 1000000, 10000000, 50000000, 100000000, 
                   500000000, 1000000000]

bucket = (companies
          .funding_total_usd
          .bucket(funding_buckets, include_over=True))
bucket.value_counts()

In [ ]:
bucket_names = ['0 to 1m', '1m to 10m', '10m to 50m', 
                '50m to 100m', '100m to 500m',
                '500m to 1b', 'Over 1b']

counts = bucket.name('bucket').value_counts()
labeled = counts.bucket.label(bucket_names)
with_names = counts.mutate(bucket_name=labeled)
with_names

In [ ]:
metrics = (companies.group_by(bucket.name('bucket'))
           .aggregate(count=companies.count(),
                      total_funding=companies.funding_total_usd.sum())
           .mutate(bucket_name=lambda x: x.bucket.label(bucket_names)))
metrics

In [ ]:
joined = (companies.mutate(bucket=bucket,
                           status=companies.status.fillna('Unknown'))
          [(companies.founded_at > '2010-01-01') |
           companies.founded_at.isnull()]
          .group_by(['bucket', 'status'])
          .size()
          .mutate(bucket_name=lambda x: (x.bucket.label(bucket_names)
                                         .fillna('Unknown'))))

table = joined.execute()

table.set_index(['status', 'bucket', 'bucket_name'])['count'].unstack('status')

## Generated SQL Queries

In [ ]:
print(ibis.impala.compile(joined))

In [ ]:
# remove the database
!rm crunchbase.db

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@pqp.io">team@tpq.io</a>

**Quant Platform** |
<a href="http://quant-platform.com">http://quant-platform.com</a>

**datapark.io** |
<a href="http://datapark.io">http://datapark.io</a>

**Python for Finance** |
<a href="http://python-for-finance.com" target="_blank">Python for Finance @ O'Reilly</a>

**Derivatives Analytics with Python** |
<a href="http://derivatives-analytics-with-python.com" target="_blank">Derivatives Analytics @ Wiley Finance</a>